In [23]:
import os
import re
import random
import gensim
import numpy as np
import pandas as pd
import nltk
from six import iteritems
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
from sklearn.cluster import SpectralClustering
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import cluster

In [2]:
from Litho.nlp_funcs import *
from Litho.similarity import (check_similarity, match_lithcode, jaccard_similarity, 
                              calc_similarity_score, print_sim_compare, merge_similar_words)

C:\Users\ifue3702\AppData\Local\Continuum\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [257]:
def labeledSentences(tokenizedDescriptions, labels = None):
    sentences = [gensim.models.doc2vec.TaggedDocument(words = i, tags = [labels[n]]) for n, i in enumerate(tokenizedDescriptions)]
    return sentences

In [3]:
root = os.getcwd()+'/'
file = 'reclasified.csv'

DF = pd.read_csv(root+file)

In [379]:
stopa = nltk.corpus.stopwords.words('english')

In [396]:
stopw2 = ['redish', 'reddish', 'red', 'black', 'blackish', 'brown', 'brownish',
          'blue', 'blueish', 'orange', 'orangeish', 'gray', 'grey', 'grayish',
          'greyish', 'white', 'whiteish', 'purple', 'purpleish', 'yellow',
          'yellowish', 'green', 'greenish', 'light', 'very', 'pink', 'coloured', 'multicoloured',
          'dark', 'color', 'colour', 'hard', 'soft', 'water', 'supply', 'fine', 'coarse',
          'medium', 'bearing', 'pipe', 'sticky', 'tough', 'small', 'stiff', 
          'running', 'streaks', 'nominal', 'bands', 'back', 'slippery', 'loose', 
          'broken', 'fractured', 'surface', 'soak', 'creek', 'ridge']


stopa= stopa+stopw2  # add the additional stopwords above

In [38]:
# stopw2 = ['redish', 'reddish', 'red', 'black', 'blackish', 'brown', 'brownish',
#           'blue', 'blueish', 'orange', 'orangeish', 'gray', 'grey', 'grayish',
#           'greyish', 'white', 'whiteish', 'purple', 'purpleish', 'yellow',
#           'yellowish', 'green', 'greenish', 'light', 'very', 'pink', 'coloured', 'multicoloured',
#           'dark', 'color', 'colour']
# stopa= stopa+stopw2  # add the additional stopwords above

In [5]:
DF = DF[['Description']]
DF = DF.dropna(how = 'any')
Descriptions=DF.Description.tolist()
Descriptions = [str(n) for n in Descriptions]

#use extend so it's a big flat list of vocab
totalvocab_lemmatized = []
totalvocab_tokenized = []
for i in Descriptions:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    totalvocab_lemmatized.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    totalvocab_tokenized.extend([allwords_tokenized])

print(np.shape(totalvocab_tokenized),  np.shape(totalvocab_lemmatized))

(130085,) (130085,)


In [7]:
###here just tokens or stemmed tokens can be used alternatively
dictionary = gensim.corpora.Dictionary(totalvocab_lemmatized)
## newt two lines remove tokens contained just once in the dictionary
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(once_ids)

dictionary.compactify()
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in totalvocab_lemmatized]
print('length corpus', len(corpus))

Dictionary(1492 unique tokens: ['quartzite', 'abundant', 'mineral', 'full', 'back']...)
length corpus 130085


In [397]:
Namoi = 'reclasified.csv'
Data = pd.read_csv(root+Namoi)
DataSampled = Data.sample(1000)
DataDes=DataSampled.Description.tolist()
DataDes = [str(n) for n in DataDes]


# #use extend so it's a big flat list of vocab
lemmatized = []
tokenized = []
for i in DataDes:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    lemmatized.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    tokenized.extend([allwords_tokenized])

print(np.shape(tokenized),  np.shape(lemmatized))


(1000,) (1000,)


In [398]:
%time tfidf = gensim.models.TfidfModel(corpus, normalize = True)
corpus_tfidf = tfidf[corpus]

Wall time: 479 ms


In [400]:
tfidfSim = gensim.similarities.Similarity(root+'tmp/',corpus_tfidf, len(dictionary))

In [401]:
# tfidfSim = gensim.similarities.Similarity()
# tfidfSimMatrix =[]
simsss = []
for n, i in enumerate(lemmatized):
    vec_bow = dictionary.doc2bow(i)
    vec_tfidf = tfidf[vec_bow]
    sims1 = tfidfSim[vec_tfidf]
    simsss.append(sims1)
#     print(sims1)
#     tfidfSimMatrix.append(sims1)
# tfidfDisMatrix = 1- np.array(tfidfSimMatrix) 


In [406]:
DataSampled['sim'] = simsss
X = np.vstack(DataSampled.sim)
kmeans = MiniBatchKMeans(n_clusters = 90, batch_size = 100).fit(X)
pred_classes = kmeans.predict(X)

print(metrics.adjusted_rand_score(DataSampled.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(DataSampled.OWN.tolist(), pred_classes.tolist()))

0.714734806621 0.652865654154


In [407]:
metrics.completeness_score(DataSampled.OWN.tolist(), pred_classes.tolist())

0.70961209696296956

In [404]:
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

In [405]:
lsiSim = gensim.similarities.Similarity(root+'tmp/',lsi[corpus_tfidf], len(dictionary))
simsss =[]
for n in lemmatized:
    vec_bow = dictionary.doc2bow(n)
    vec_lsi = lsi[vec_bow]
    sims1 = lsiSim[vec_lsi]
    simsss.append(sims1)
# lsiDisMatrix = 1- np.array(lsiSimMatrix) 

In [26]:
lda = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=50, chunksize=10000, passes=10, workers = 8)

In [27]:
ldaSim = gensim.similarities.Similarity(root+'tmp/', lda[corpus], len(dictionary))
simsss =[]
for n in lemmatized:
    vec_bow = dictionary.doc2bow(n)
    vec_lda = lda[vec_bow]
    sims1 = ldaSim[vec_lda]
    simsss.append(sims1)

In [408]:
size = 1000
labs = list(range(len(totalvocab_lemmatized)))
labs = ['lab_{}'.format(n) for n in labs]
taggedDescriptions = labeledSentences(totalvocab_lemmatized, labels = labs)
doc2vec = gensim.models.Doc2Vec(size = size, window = 13, workers = 8, seed = 1, 
                                min_count = 5, sample = 0.1e-5, dm = 1, dm_concat = 1, 
                                dm_mean = 1, alpha = 0.25)
doc2vec.build_vocab(taggedDescriptions)


In [409]:
vectors = []
for n in lemmatized:
    vectors.append(np.reshape(doc2vec.infer_vector(n), (1,size)))
vectors = np.reshape(np.asarray(vectors), (1000, size))
kmeans = MiniBatchKMeans(n_clusters = 90, batch_size = 100).fit(vectors)
pred_classes = kmeans.predict(vectors)

print(metrics.adjusted_rand_score(DataSampled.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(DataSampled.OWN.tolist(), pred_classes.tolist()),
      metrics.completeness_score(DataSampled.OWN.tolist(), pred_classes.tolist()))

0.434457623617 0.608323503602 0.665401278107


In [411]:
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool


def plot_embedding(arr, labels=None, filename='plot.html'):
    """Short summary.

    Parameters
    ----------
    arr : np.array
       2D array with embeddings.
    labels : list(str)
       List of labels (the default is None).
    filename : str
       Path to the output html file containing the plot (the default is 'plot.html').

    """
    tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(arr)

    plot_data = {
       'x': tsne_results[:, 0],
       'y': tsne_results[:, 1]
    }

    plot_tools = ['pan', 'zoom_in', 'zoom_out']

    if labels is not None:
        plot_data['label'] = labels

        hover = HoverTool(tooltips=[
           ("Description", "@label")
        ])

        plot_tools.append(hover)

    to_plot = ColumnDataSource(plot_data)
    p = figure(title="Descriptions t-SNE", tools=plot_tools)
    p.xaxis.axis_label = ''
    p.yaxis.axis_label = ''

    p.circle('x', 'y', source=to_plot,
            fill_alpha=0.8, size=10)

    # p.add_layout(labels)
    output_file(filename)
    show(p)
    
plot_embedding([doc2vec.infer_vector(x) for x in lemmatized[:1000]],
               [';'.join(x) for x in lemmatized[:1000]])

In [43]:
from nltk.corpus import wordnet as wn

In [233]:
clay = wn.synset('clay.n.01')
mud = wn.synset('mud.n.01')
# pug = wn.synset('pug.n.02')
sand = wn.synset('sand.n.01')
silt = wn.synset('silt.n.01')
gravel =  wn.synset('gravel.n.01')
stone = wn.synset('stone.n.01')
pebble = wn.synset('pebble.n.01')
boulder = wn.synset('boulder.n.01')
granite = wn.synset('granite.n.01')
basalt = wn.synset('basalt.n.01')
lava = wn.synset('lava.n.01')
tuff = wn.synset('tuff.n.01')
volcanic = wn.synset('volcanic.s.03')
rhyolite = wn.synset('rhyolite.n.01')
breccia = wn.synsets('breccia')[0]
agglomerate = wn.synset('agglomerate.n.01')
zeolite = wn.synset('zeolite.n.01')
andesite = wn.synset('andesite.n.01')
diorite = wn.synset('diorite.n.01')
porphyry = wn.synset('porphyry.n.01')
# dolerite = wn.synset('dolerite.n.01')
feldspar = wn.synset('feldspar.n.01')
igneous = wn.synset('igneous.a.01')

sandstone = wn.synset('sandstone.n.01')
# greywacke = wn.synset('greywacke.n.01')
# arkose = wn.synset('arkose.n.01')

shale = wn.synset('shale.n.01')
claystone = wn.synset('claystone.n.01')
# mudstone = wn.synset('mudstone.n.01')
siltstone = wn.synset('siltstone.n.01')

limestone =  wn.synset('limestone.n.01')
dolomite = wn.synset('dolomite.n.01')
# calcrete = wn.synset('calcrete.n.01')
siderite = wn.synset('siderite.n.01')
chalk = wn.synset('chalk.n.01')
marl = wn.synset('marl.n.01')
calcite = wn.synset('calcite.n.01')

mica = wn.synset('mica.n.01')
schist = wn.synset('schist.n.01')
# serpentine = wn.synset('serpentine.n.01')
soapstone = wn.synset('soapstone.n.01')
slate = wn.synset('slate.n.03')
# phyllite  = wn.synset('phyllite.n.01')

carbonaceous = wn.synset('carbonaceous.a.01')
coal = wn.synset('coal.n.01')
lignite = wn.synset('lignite.n.01')
wood = wn.synset('wood.n.01')

jasper = wn.synset('jasper.n.01')
chert = wn.synset('chert.n.01')
# silcrete = wn.synset('silcrete.n.01')
laterite = wn.synset('laterite.n.01')
# ironstone = wn.synset('ironstone.n.01')
cement = wn.synset('cement.n.03')
# blueMetal = wn.synset('wood.n.01')

soil = wn.synset('soil.n.02')
loam = wn.synset('loam.n.01')
wood = wn.synset('wood.n.01')

bentonite = wn.synset('bentonite.n.01')

conglomerate = wn.synset('conglomerate.n.01')

drift = wn.synset('drift.n.04')

quartz = wn.synset('quartz.n.02')

rock = wn.synset('rock.n.01')
alluvium = wn.synset('alluvium.n.01')
water = wn.synset('water.n.01')
cavity = wn.synset('cavity.n.01')
sedimentary = wn.synset('sedimentary.a.01')
channel = wn.synset('channel.n.02')
ground = wn.synset('ground.n.07')
earth = wn.synset('earth.n.02')

sandy = wn.synsets('sandy', pos = wn.ADJ)[1]
clayey = wn.synsets('clayey', pos = wn.ADJ)[0]
silty = wn.synsets('silty')[0]
gravelly = wn.synset('gravelly.s.01')
stony = wn.synsets('stony')[0]
pebbly = wn.synsets('pebbly')[0]
gritty = wn.synsets('gritty')[0]



# for n in drift:
#     print(n.definition())
print(earth)
print(earth.definition())
# sandy = wngreywacke.synset('sandy.a.01')
# for n in sandy:
#     print(n.definition())
# print(sandy)

Synset('earth.n.02')
the loose soft material that makes up a large part of the land surface


In [176]:

Lemma = nltk.WordNetLemmatizer()
print(Lemma.lemmatize('carbonaceous'))

carbonaceous


In [247]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfidf = vect.fit_transform([n.lower() for n in Descriptions])

In [248]:
kmeans = MiniBatchKMeans(n_clusters = 90, batch_size = 100, ).fit(tfidf)
pred_classes = kmeans.predict(tfidf)

print(metrics.completeness_score(Data.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_rand_score(Data.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(Data.OWN.tolist(), pred_classes.tolist()))

0.404928867022 0.232049174226 0.401791987721


In [412]:
from __future__ import division
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import math
import numpy as np
import sys


In [413]:
# Parameters to the algorithm. Currently set to values that was reported
# in the paper to produce "best" results.
ALPHA = 0.2
BETA = 0.45
ETA = 0.4
PHI = 0.2
DELTA = 0.85

brown_freqs = dict()
N = 0

In [450]:
######################### word similarity ##########################

def get_best_synset_pair(word_1, word_2):
    """ 
    Choose the pair with highest path similarity among all pairs. 
    Mimics pattern-seeking behavior of humans.
    """
    max_sim = -1.0
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    if len(synsets_1) == 0 or len(synsets_2) == 0:
        return None, None
    else:
        max_sim = -1.0
        best_pair = None, None
        for synset_1 in synsets_1:
            for synset_2 in synsets_2:
                sim = wn.path_similarity(synset_1, synset_2)
                if sim is not None and sim > max_sim:
                    if sim > max_sim:
                        max_sim = sim
                        best_pair = synset_1, synset_2
        return best_pair

def length_dist(synset_1, synset_2):
    """
    Return a measure of the length of the shortest path in the semantic
    ontology (Wordnet in our case as well as the paper's) between two
    synsets.
    """
    l_dist = sys.maxsize
    if synset_1 is None or synset_2 is None:
        return 0.0
    if synset_1 == synset_2:
        # if synset_1 and synset_2 are the same synset return 0
        l_dist = 0.0
    else:
        wset_1 = set([str(x.name()) for x in synset_1.lemmas()])
        wset_2 = set([str(x.name()) for x in synset_2.lemmas()])
        if len(wset_1.intersection(wset_2)) > 0:
            # if synset_1 != synset_2 but there is word overlap, return 1.0
            l_dist = 1.0
        else:
            # just compute the shortest path between the two
            l_dist = synset_1.shortest_path_distance(synset_2)
            if l_dist is None:
                l_dist = 0.0
    # normalize path length to the range [0,1]
    return math.exp(-ALPHA * l_dist) 

def hierarchy_dist(synset_1, synset_2):
    """
    Return a measure of depth in the ontology to model the fact that 
    nodes closer to the root are broader and have less semantic similarity
    than nodes further away from the root.
    """
    h_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return h_dist
    if synset_1 == synset_2:
        # return the depth of one of synset_1 or synset_2
        h_dist = max([x[1] for x in synset_1.hypernym_distances()])
    else:
        # find the max depth of least common subsumer
        hypernyms_1 = {x[0]:x[1] for x in synset_1.hypernym_distances()}
        hypernyms_2 = {x[0]:x[1] for x in synset_2.hypernym_distances()}
        lcs_candidates = set(hypernyms_1.keys()).intersection(
            set(hypernyms_2.keys()))
        if len(lcs_candidates) > 0:
            lcs_dists = []
            for lcs_candidate in lcs_candidates:
                lcs_d1 = 0
                if lcs_candidate in hypernyms_1:
                    lcs_d1 = hypernyms_1[lcs_candidate]
                lcs_d2 = 0
                if lcs_candidate in hypernyms_2:
                    lcs_d2 = hypernyms_2[lcs_candidate]
                lcs_dists.append(max([lcs_d1, lcs_d2]))
            h_dist = max(lcs_dists)
        else:
            h_dist = 0
    return ((math.exp(BETA * h_dist) - math.exp(-BETA * h_dist)) / 
        (math.exp(BETA * h_dist) + math.exp(-BETA * h_dist)))
    
def word_similarity(word_1, word_2):
    synset_pair = get_best_synset_pair(word_1, word_2)
    return (length_dist(synset_pair[0], synset_pair[1]) * 
        hierarchy_dist(synset_pair[0], synset_pair[1]))

In [452]:
######################### sentence similarity ##########################

def most_similar_word(word, word_set):
    """
    Find the word in the joint word set that is most similar to the word
    passed in. We use the algorithm above to compute word similarity between
    the word and each word in the joint word set, and return the most similar
    word and the actual similarity value.
    """
    max_sim = -1.0
    sim_word = ""
    for ref_word in word_set:
        sim = word_similarity(word, ref_word)
        if sim > max_sim:
            max_sim = sim
            sim_word = ref_word
        return sim_word, max_sim
    
def info_content(lookup_word):
    """
    Uses the Brown corpus available in NLTK to calculate a Laplace
    smoothed frequency distribution of words, then uses this information
    to compute the information content of the lookup_word.
    """
    global N
    if N == 0:
        # poor man's lazy evaluation
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if word not in brown_freqs:
                    brown_freqs[word] = 0
                brown_freqs[word] = brown_freqs[word] + 1
                N = N + 1
    lookup_word = lookup_word.lower()
    n = 0 if lookup_word not in brown_freqs else brown_freqs[lookup_word]
    return 1.0 - (math.log(n + 1) / math.log(N + 1))
    
def semantic_vector(words, joint_words, info_content_norm):
    """
    Computes the semantic vector of a sentence. The sentence is passed in as
    a collection of words. The size of the semantic vector is the same as the
    size of the joint word set. The elements are 1 if a word in the sentence
    already exists in the joint word set, or the similarity of the word to the
    most similar word in the joint word set if it doesn't. Both values are 
    further normalized by the word's (and similar word's) information content
    if info_content_norm is True.
    """
    sent_set = set(words)
    semvec = np.zeros(len(joint_words))
    i = 0
    for joint_word in joint_words:
        if joint_word in sent_set:
            # if word in union exists in the sentence, s(i) = 1 (unnormalized)
            semvec[i] = 1.0
            if info_content_norm:
                semvec[i] = semvec[i] * math.pow(info_content(joint_word), 2)
        else:
            # find the most similar word in the joint set and set the sim value
            sim_word, max_sim = most_similar_word(joint_word, sent_set)
            semvec[i] = PHI if max_sim > PHI else 0.0
            if info_content_norm:
                semvec[i] = semvec[i] * info_content(joint_word) * info_content(sim_word)
        i = i + 1
    return semvec                
            
def semantic_similarity(sentence_1, sentence_2, info_content_norm):
    """
    Computes the semantic similarity between two sentences as the cosine
    similarity between the semantic vectors computed for each sentence.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = set(words_1).union(set(words_2))
    vec_1 = semantic_vector(words_1, joint_words, info_content_norm)
    vec_2 = semantic_vector(words_2, joint_words, info_content_norm)
    return np.dot(vec_1, vec_2.T) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))

In [416]:
######################### word order similarity ##########################

def word_order_vector(words, joint_words, windex):
    """
    Computes the word order vector for a sentence. The sentence is passed
    in as a collection of words. The size of the word order vector is the
    same as the size of the joint word set. The elements of the word order
    vector are the position mapping (from the windex dictionary) of the 
    word in the joint set if the word exists in the sentence. If the word
    does not exist in the sentence, then the value of the element is the 
    position of the most similar word in the sentence as long as the similarity
    is above the threshold ETA.
    """
    wovec = np.zeros(len(joint_words))
    i = 0
    wordset = set(words)
    for joint_word in joint_words:
        if joint_word in wordset:
            # word in joint_words found in sentence, just populate the index
            wovec[i] = windex[joint_word]
        else:
            # word not in joint_words, find most similar word and populate
            # word_vector with the thresholded similarity
            sim_word, max_sim = most_similar_word(joint_word, wordset)
            if max_sim > ETA:
                wovec[i] = windex[sim_word]
            else:
                wovec[i] = 0
        i = i + 1
    return wovec

def word_order_similarity(sentence_1, sentence_2):
    """
    Computes the word-order similarity between two sentences as the normalized
    difference of word order between the two sentences.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = list(set(words_1).union(set(words_2)))
    windex = {x[1]: x[0] for x in enumerate(joint_words)}
    r1 = word_order_vector(words_1, joint_words, windex)
    r2 = word_order_vector(words_2, joint_words, windex)
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))

In [417]:
######################### overall similarity ##########################

def similarity(sentence_1, sentence_2, info_content_norm):
    """
    Calculate the semantic similarity between two sentences. The last 
    parameter is True or False depending on whether information content
    normalization is desired or not.
    """
    return DELTA * semantic_similarity(sentence_1, sentence_2, info_content_norm) + \
        (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2)

In [453]:
######################### main / test ##########################

# the results of the algorithm are largely dependent on the results of 
# the word similarities, so we should test this first...
# word_pairs = [
#   ["asylum", "fruit", 0.21],
#   ["autograph", "shore", 0.29],
#   ["autograph", "signature", 0.55],
#   ["automobile", "car", 0.64],
#   ["bird", "woodland", 0.33],
#   ["boy", "rooster", 0.53],
#   ["boy", "lad", 0.66],
#   ["boy", "sage", 0.51],
#   ["cemetery", "graveyard", 0.73],
#   ["coast", "forest", 0.36],
#   ["coast", "shore", 0.76],
#   ["cock", "rooster", 1.00],
#   ["cord", "smile", 0.33],
#   ["cord", "string", 0.68],
#   ["cushion", "pillow", 0.66],
#   ["forest", "graveyard", 0.55],
#   ["forest", "woodland", 0.70],
#   ["furnace", "stove", 0.72],
#   ["glass", "tumbler", 0.65],
#   ["grin", "smile", 0.49],
#   ["gem", "jewel", 0.83],
#   ["hill", "woodland", 0.59],
#   ["hill", "mound", 0.74],
#   ["implement", "tool", 0.75],
#   ["journey", "voyage", 0.52],
#   ["magician", "oracle", 0.44],
#   ["magician", "wizard", 0.65],
#   ["midday", "noon", 1.0],
#   ["oracle", "sage", 0.43],
#   ["serf", "slave", 0.39]
# ]
# for word_pair in word_pairs:
#     print("%s\t%s\t%.2f\t%.2f" % (word_pair[0], word_pair[1], word_pair[2],
#                                   word_similarity(word_pair[0], word_pair[1])))





sentence_pairs = [
    ["I like that bachelor.", "I like that unmarried man.", 0.561],
    ["John is very nice.", "Is John very nice?", 0.977],
    ["Red alcoholic drink.", "A bottle of wine.", 0.585],
    ["Red alcoholic drink.", "Fresh orange juice.", 0.611],
    ["Red alcoholic drink.", "An English dictionary.", 0.0],
    ["Red alcoholic drink.", "Fresh apple juice.", 0.420],
    ["A glass of cider.", "A full cup of apple juice.", 0.678],
    ["It is a dog.", "That must be your dog.", 0.739],
    ["It is a dog.", "It is a log.", 0.623],
    ["It is a dog.", "It is a pig.", 0.790],
    ["Dogs are animals.", "They are common pets.", 0.738],
    ["Canis familiaris are animals.", "Dogs are common pets.", 0.362],
    ["I have a pen.", "Where do you live?", 0.0],
    ["I have a pen.", "Where is ink?", 0.129],
    ["I have a hammer.", "Take some nails.", 0.508],
    ["I have a hammer.", "Take some apples.", 0.121]
]
for sent_pair in sentence_pairs:
    print("%s\t%s\t%.3f\t%.3f\t%.3f" % (sent_pair[0], sent_pair[1], sent_pair[2], 
        similarity(sent_pair[0], sent_pair[1], False),
        similarity(sent_pair[0], sent_pair[1], True)))

I like that bachelor.	I like that unmarried man.	0.561	0.788	0.319
John is very nice.	Is John very nice?	0.977	0.615	0.903
Red alcoholic drink.	A bottle of wine.	0.585	0.256	0.069
Red alcoholic drink.	Fresh orange juice.	0.611	0.268	0.058
Red alcoholic drink.	An English dictionary.	0.000	0.284	0.074
Red alcoholic drink.	Fresh apple juice.	0.420	0.252	0.042
A glass of cider.	A full cup of apple juice.	0.678	0.626	0.334
It is a dog.	That must be your dog.	0.739	0.422	0.682
It is a dog.	It is a log.	0.623	0.808	0.257
It is a dog.	It is a pig.	0.790	0.784	0.188
Dogs are animals.	They are common pets.	0.738	0.420	0.078
Canis familiaris are animals.	Dogs are common pets.	0.362	0.394	0.070
I have a pen.	Where do you live?	0.000	0.101	0.120
I have a pen.	Where is ink?	0.129	0.100	0.046
I have a hammer.	Take some nails.	0.508	0.256	0.070
I have a hammer.	Take some apples.	0.121	0.293	0.138
